In [1]:
!curl ifconfig.co/ip
!curl ifconfig.co/country
!curl ifconfig.co/city

34.123.96.115
United States
Council Bluffs


In [ ]:
# import pandas as pd #@param
# sheet_id = "1JzkceRcggRAaQJE-hekm7oexJv4bdQ67IKXiO81Qh4U"
# sheet_name = "System Config"
# sheet_name = sheet_name.replace(' ', '%20')
# csvUrl = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# pd.read_csv(csvUrl)

In [ ]:
systemNames = "39-DGG,3USX-F,6RQ9-A,K42-IE,SV-K8J,VSJ-PP,4-CUM5,8MG-J6,BND-16,BWF-ZZ,IOO-7O,RLSI-V,04-LQM,LX-ZOJ,O-VWPB, A-YB15,AU2V-J,D-6PKO,QZX-L9,SG-3HY,SY-0AM,BM-VYZ,EPCD-D,L-TLFU,MN9P-A,Q-GICU,RAI-0E,TA9T-P,1S-SU1,42G-OB,G4-QU6,HD-HOZ,LEM-I1,ND-GL4,V2-GZS,BGMZ-0,FZX-PU,GQ-7SP,I2D3-5,O9K-FT,R1O-GN,RQOO-U,0S1-GI,74-DRC,M3-H2Y,O31W-6,6FS-CZ,B3ZU-H,C3J0-O,G-KCFT,GSO-SR,H7S-5I,HPV-RJ,WNM-V0,9-0QB7,HVGR-R,K76A-3,K95-9I,M-75WN,PNFW-O, TDP-T3,7M4-4C,MJ-5F9,Z-ENUD" #@param {type:"string"}
systemNames = systemNames.replace(' ', '').split(',')
systemNames


In [72]:
# 30000142 jita

import concurrent.futures, requests, pytz
import pandas as pd
from datetime import datetime


# UTC timezone Datetime
dt_local = datetime.now(pytz.utc)
# Convert UTC timezone to Asia/Dhaka timezone
dt_bd = dt_local.astimezone(pytz.timezone('Asia/Dhaka'))
time = dt_bd.strftime("%Y%m%d_%I%M%S%P")



HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
            'Accept-Language': 'en-US;q=0.9'})

# systems IDs
URL = f"https://esi.evetech.net/latest/universe/ids/?datasource=tranquility&language=en"
res = requests.post(URL, headers=HEADERS, json = systemNames)
nameIds = res.json()

systems = [sys['id'] for sys in nameIds['systems']]
print(f"Total Systems: {len(systems)} \n")


URL = f"https://esi.evetech.net/latest/universe/system_kills/?datasource=tranquility"
res = requests.get(URL, headers=HEADERS)
kills = res.json()
kills_df = pd.DataFrame(kills, index=None)


URL = f"https://esi.evetech.net/latest/universe/system_jumps/?datasource=tranquility"
res = requests.get(URL, headers=HEADERS)
jumps = res.json()
jumps_df = pd.DataFrame(jumps, index=None)




def check_info(s):
  system_id = s

  URL = f"https://esi.evetech.net/latest/universe/systems/{s}/?datasource=tranquility&language=en"
  res = requests.get(URL, headers=HEADERS)
  system_info = res.json()
  # print(s)
  name = system_info['name']
  security_status = system_info['security_status']
  # print(name, '\n')

  # Sometimes 'security_class' isn't indcluded
  try:
    security_class = system_info['security_class']
  except:
    security_class = 'NULL'

  # kills_df comes from before
  system_kills = kills_df.loc[kills_df['system_id'] == s]
  if len(system_kills.index):
    npc_kills = system_kills.iloc[0]['npc_kills']
    pod_kills = system_kills.iloc[0]['pod_kills']
    ship_kills = system_kills.iloc[0]['ship_kills']
    # system_id = system_kills.iloc[0]['system_id']
  else:
    npc_kills = 0
    pod_kills = 0
    ship_kills = 0



  # jumps_df comes from before
  system_jump = jumps_df.loc[jumps_df['system_id'] == s]
  if len(system_jump.index):
    ship_jumps = system_jump.iloc[0]['ship_jumps']
  else:
    ship_jumps = 0





  dict1 =  {'system_id': system_id,
            'name': name,
            'security_class': security_class,
            'security_status': security_status,
            # 'security_status': round(security_status, 3),
            'ship_jumps': ship_jumps,
            'npc_kills': npc_kills,
            'ship_kills': ship_kills,
            'pod_kills': pod_kills,
            }

  return dict1


Total Systems: 64 



In [ ]:
%%time
final_df = pd.DataFrame()

with concurrent.futures.ProcessPoolExecutor(max_workers=15) as executor:
  for a in executor.map(check_info, systems):
  # for a in executor.map(check_info, notDone):
    df = pd.DataFrame([a])
    final_df = pd.concat([final_df,df], ignore_index=True)
    # print(a)

In [75]:
print(time)
final_df


20230625_124929pm


system_id    name security_class  security_status  ship_jumps  npc_kills  \
0    30002874  R1O-GN             K1        -0.125715         133         33   
1    30005089  TDP-T3             K5        -0.475821         149        411   
2    30005138  7M4-4C             K5        -0.468539          18        115   
3    30005133  MJ-5F9             K6        -0.619503         481       1075   
4    30005132  Z-ENUD             K5        -0.504997         219        632   
..        ...     ...            ...              ...         ...        ...   
59   30002871  HVGR-R             K6        -0.601868           5         14   
60   30002872  K76A-3             K7        -0.821322           4        479   
61   30002873  K95-9I             K6        -0.646943           3        606   
62   30002869  M-75WN             K5        -0.499043          51          0   
63   30002870  PNFW-O             K5        -0.539760          34         66   

    ship_kills  pod_kills  
0            0          0  
1            0          0  
2            0          0  
3            5          3  
4            2          0  
..         ...        ...  
59           0          0  
60           0          0  
61           0          0  
62           0          0  
63           0          0  

[64 rows x 8 columns]

In [ ]:
print(time)

final_df
# final

In [ ]:
# CSV save

final_df.to_csv(f"/content/drive/MyDrive/00_Sheets/eve_security_Kills_{time}.csv", encoding = "utf-8",  index=False)

# final.to_csv(f"/content/drive/MyDrive/00_Sheets/EVE__all_systems_names+security+jumpsFromJita.csv", encoding = "utf-8",  index=False)
# final.to_csv(f"/content/drive/MyDrive/00_Sheets/EVE_jumps_from_Jita_to_all_systems.csv", encoding = "utf-8",  index=False)

In [ ]:
done = list(final_df['system_id'])
notDone = [ele for ele in systems if ele not in done]
len(notDone)